In [1]:
"""
A convenience script to playback random demonstrations from
a set of demonstrations stored in a hdf5 file.

Arguments:
    --folder (str): Path to demonstrations
    --use-actions (optional): If this flag is provided, the actions are played back
        through the MuJoCo simulator, instead of loading the simulator states
        one by one.
    --visualize-gripper (optional): If set, will visualize the gripper site

Example:
    $ python playback_demonstrations_from_hdf5.py --folder ../models/assets/demonstrations/lift/
    python playback_demonstrations_from_hdf5.py --folder  /home/ns/robosuite_git/lift_demo
"""

import argparse
import json
import os
import random

import h5py
import numpy as np

import robosuite
import imageio

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--folder",
    type=str,
    help="Path to your demonstration folder that contains the demo.hdf5 file, e.g.: "
    "'path_to_assets_dir/demonstrations/YOUR_DEMONSTRATION'",
),
parser.add_argument(
    "--use-actions",
    action="store_true",
)
args = parser.parse_args([])

In [3]:
args.folder='/home/ns/robosuite_git/lift_demo'
args.folder

'/home/ns/robosuite_git/lift_demo'

In [4]:
args.use_actions=True

In [10]:

demo_path = args.folder
hdf5_path = os.path.join(demo_path, "demo.hdf5")
f = h5py.File(hdf5_path, "r")
env_name = f["data"].attrs["env"]
env_info = json.loads(f["data"].attrs["env_info"])

# env = robosuite.make(
#     **env_info,
#     has_renderer=True,
#     has_offscreen_renderer=False,
#     ignore_done=True,
#     use_camera_obs=False,
#     reward_shaping=True,
#     control_freq=20,
# )

env = robosuite.make(
    **env_info,
    ignore_done=True,
    use_camera_obs=True,
    has_renderer=False,
    has_offscreen_renderer=True,
    render_camera="frontview",
    control_freq=20,
)


# list of all demonstrations episodes
demos = list(f["data"].keys())


print("Playing back random episode... (press ESC to quit)")

# select an episode randomly
ep = random.choice(demos)

# read the model xml, using the metadata stored in the attribute for this episode
model_xml = f["data/{}".format(ep)].attrs["model_file"]

env.reset()
xml = env.edit_model_xml(model_xml)
env.reset_from_xml_string(xml)
env.sim.reset()
env.viewer.set_camera(0)

# load the flattened mujoco states
states = f["data/{}/states".format(ep)][()]

if args.use_actions:

    # load the initial state
    env.sim.set_state_from_flattened(states[0])
    env.sim.forward()

    # load the actions and play them back open-loop
    actions = np.array(f["data/{}/actions".format(ep)][()])
    num_actions = actions.shape[0]

    for j, action in enumerate(actions):
        env.step(action)
        env.render()

        if j < num_actions - 1:
            # ensure that the actions deterministically lead to the same recorded states
            state_playback = env.sim.get_state().flatten()
            if not np.all(np.equal(states[j + 1], state_playback)):
                err = np.linalg.norm(states[j + 1] - state_playback)
                print(f"[warning] playback diverged by {err:.2f} for ep {ep} at step {j}")

else:

    # force the sequence of internal mujoco states one by one
    for state in states:
        env.sim.set_state_from_flattened(state)
        env.sim.forward()
        env.render()

f.close()

Playing back random episode... (press ESC to quit)


AttributeError: 'NoneType' object has no attribute 'set_camera'

In [6]:
frame=env.render(mode="rgb_array", height=512, width=512, camera_name="frontview")

TypeError: render() got an unexpected keyword argument 'mode'

In [9]:
env.close()

In [12]:
action.shape

(7,)

In [4]:
args.use_actions

False

In [5]:
demo_path = args.folder
hdf5_path = os.path.join(demo_path, "demo.hdf5")
f = h5py.File(hdf5_path, "r")
env_name = f["data"].attrs["env"]
env_info = json.loads(f["data"].attrs["env_info"])

# env = robosuite.make(
#     **env_info,
#     has_renderer=True,
#     has_offscreen_renderer=False,
#     ignore_done=True,
#     use_camera_obs=False,
#     reward_shaping=True,
#     control_freq=20,
#     render_camera="frontview",
# ) 

In [6]:
# env = robosuite.make(
#     "Lift",
#     robots="Sawyer",
#     ignore_done=True,
#     use_camera_obs=False,
#     has_renderer=True,
#     has_offscreen_renderer=False,
#     control_freq=20,
# )


env = robosuite.make(
    "Lift",
    robots="Sawyer",
    ignore_done=True,
    use_camera_obs=True,
    has_renderer=False,
    has_offscreen_renderer=True,
    render_camera="frontview",
    control_freq=20,
)

In [7]:
demos = list(f["data"].keys()) 
ep = random.choice(demos)
ep

'demo_11'

In [9]:
# read the model xml, using the metadata stored in the attribute for this episode
model_xml = f["data/{}".format(ep)].attrs["model_file"]

env.reset()
xml = env.edit_model_xml(model_xml)
env.reset_from_xml_string(xml)
env.sim.reset()
# env.viewer.set_camera(0)

# load the flattened mujoco states
states = f["data/{}/states".format(ep)][()]

In [16]:
actions = np.array(f["data/{}/actions".format(ep)][()])
num_actions = actions.shape[0]
actions.shape

(431, 7)

In [17]:
obs = env.reset()
obs.keys()

odict_keys(['robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'agentview_image', 'cube_pos', 'cube_quat', 'gripper_to_cube_pos', 'robot0_proprio-state', 'object-state'])

In [19]:
# load the initial state
env.sim.set_state_from_flattened(states[0])
env.sim.forward()

In [20]:
action=actions[0]
env.step(action)

AssertionError: environment got invalid action dimension -- expected 8, got 7

In [10]:
writer = imageio.get_writer(f"{ep}_playback_ns.mp4", fps=20)
for state in states:
    env.sim.set_state_from_flattened(state)
    env.sim.forward()
    # env.render() 
    frame=env.render(mode="rgb_array", height=512, width=512, camera_name="frontview")
    writer.append_data(frame) 

writer.close()

TypeError: render() got an unexpected keyword argument 'mode'

In [10]:
f.close()

In [11]:
env.close()